## 对话智能体中的提示语结构

1. 确认是否新对话
2. 补充提问
    - 追加到旧的对话
    - 使用提示语构建新对话
    - 结合 init_messages 构建新对话
3. 构建对话
    - 收集最近记忆
    - 结合知识检索
    - 结合工具箱

In [3]:
from illufly.chat import FakeLLM
from illufly.types import Template

llm = FakeLLM()
llm("写一首诗词吧", verbose=True, new_chat=True)
llm.memory

[INFO] 记住 10 轮对话
[INFO] FakeLLM: [{'role': 'user', 'content': '写一首诗词吧'}]
这是一个模拟调用!

[{'role': 'user', 'content': '写一首诗词吧'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

In [4]:
llm = FakeLLM()
llm.build_chat_memory("hi")

[{'role': 'user', 'content': 'hi'}]

## 构建对话

### 新对话

In [5]:
from illufly.chat import FakeLLM
from illufly.types import Template

llm = FakeLLM()
llm("写一首诗词吧")
llm.memory

这是一个模拟调用!

[{'role': 'user', 'content': '写一首诗词吧'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

### 连续对话

In [6]:
llm("再写一首")
llm.memory

这是一个模拟调用!

[{'role': 'user', 'content': '写一首诗词吧'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'},
 {'role': 'user', 'content': '再写一首'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

### 如果使用 new_chat 标志

In [7]:
llm("再写一首", new_chat=True)
llm.memory

这是一个模拟调用!

[{'role': 'user', 'content': '再写一首'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

### 在调用时使用 system 角色的提示语

In [8]:
llm([("system", "你是一个作家"), ("user", "写一首诗")])
llm.memory

这是一个模拟调用!

[{'role': 'system', 'content': '你是一个作家'},
 {'role': 'user', 'content': '写一首诗'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

### 在调用时使用 system 角色并补充 user 消息

In [9]:
llm([("system", "你是一个作家，帮我写首诗")])
llm.memory

这是一个模拟调用!

[{'role': 'system', 'content': '你是一个作家，帮我写首诗'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

### 在声明实例时使用 system 角色的提示语

In [10]:
from illufly.chat import FakeLLM
from illufly.types import Template

llm = FakeLLM(memory=[("system", "你是一个作家")])
llm("写一首诗词吧")
llm.memory

这是一个模拟调用!

[{'role': 'system', 'content': '你是一个作家'},
 {'role': 'user', 'content': '写一首诗词吧'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

### 绑定 task 并自动补充 user 消息

In [11]:
llm = FakeLLM(memory=[("system", Template(text="你是一个作家，请帮我{{task}}"))])
llm("写一首诗词吧")
llm.memory

这是一个模拟调用!

[{'role': 'system', 'content': '你是一个作家，请帮我写一首诗词吧'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

### 绑定 task 并提供 user 消息

In [12]:
llm = FakeLLM(memory=[
    ("system", Template(text="你是一个作家，请帮我{{task}}")), 
    ("user", "开始吧")
])
llm("写一首诗词吧")
llm.memory

这是一个模拟调用!

[{'role': 'system', 'content': '你是一个作家，请帮我写一首诗词吧'},
 {'role': 'user', 'content': '开始吧'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

## 映射提示语中的模板变量

### 默认映射

In [13]:
from illufly.chat import FakeLLM
from illufly.types import Template

llm = FakeLLM(
    memory=[Template(text="请你帮我写{{task}}"), "请开始"]
)
llm("一首儿歌，4行")
llm.memory

这是一个模拟调用!

[{'role': 'system', 'content': '请你帮我写一首儿歌，4行'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

### 重新映射

In [14]:
from illufly.chat import FakeLLM
from illufly.types import Template

llm = FakeLLM(
    memory=[
        Template(text="请你帮我写{{mytask}}", lazy_binding_map={"mytask": "task"}), 
        "请开始"
    ]
)
llm("一首儿歌，4行")
llm.memory

这是一个模拟调用!

[{'role': 'system', 'content': '请你帮我写一首儿歌，4行'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

In [15]:
from illufly.chat import FakeLLM
from illufly.types import Template

llm = FakeLLM(
    memory=[
        Template(text="请你帮我写{{mytask}}", lazy_binding_map={"mytask": "task"}), 
        "请开始"
    ]
)
llm([Template(text="写一首歌{{task}}"), "一首儿歌，4行"])
llm.memory

这是一个模拟调用!

[{'role': 'system', 'content': '写一首歌一首儿歌，4行'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

## 携带背景知识

### 文本知识

In [21]:
from illufly.chat import FakeLLM
from illufly.types import Template

llm = FakeLLM()
llm.add_knowledge("illufly 是一个多智能体解决方案")
llm.add_knowledge("普鸿是做消防业务的")
llm([Template(text="已知：{{knowledge}}"), "写一首诗词吧"])
llm.memory

这是一个模拟调用!

[{'role': 'system', 'content': "已知：['普鸿是做消防业务的', 'illufly 是一个多智能体解决方案']"},
 {'role': 'user', 'content': '写一首诗词吧'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

In [22]:
llm.provider_dict

{'last_output': '这是一个模拟调用!',
 'resources': [],
 'task': '写一首诗词吧',
 'tools_name': '',
 'tools_desc': '',
 'knowledge': ['普鸿是做消防业务的', 'illufly 是一个多智能体解决方案']}

### 动态查询

## 管理记忆长度

### 默认记住10轮

In [1]:
from illufly.chat import FakeLLM
from illufly.types import Template
from illufly.io import alog, log

a = FakeLLM()

a("你能帮我写一首关于兔子做梦的四句儿歌?", verbose=True)

[INFO] 记住 10 轮对话
[INFO] FakeLLM: [{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'}]
这是一个模拟调用!

'这是一个模拟调用!'

In [2]:
await a("你能帮我写一首关于兔子做梦的四句儿歌?", handlers=[alog])

这是一个模拟调用!

'这是一个模拟调用!'

In [3]:
a.build_chat_memory("hi", remember_rounds=3)

[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'},
 {'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'},
 {'role': 'user', 'content': 'hi'}]

In [4]:
a("换成小羊吧")

这是一个模拟调用!

'这是一个模拟调用!'

### 限制记忆中的对话轮数

In [5]:
a("改为在蓝天上")

这是一个模拟调用!

'这是一个模拟调用!'

In [6]:
a("改为在水塘", remember_rounds=1, verbose=True)

[INFO] 记住 1 轮对话
[INFO] FakeLLM: [{'role': 'user', 'content': '改为在蓝天上'}, {'role': 'assistant', 'content': '这是一个模拟调用!'}, {'role': 'user', 'content': '改为在水塘'}]
这是一个模拟调用!

'这是一个模拟调用!'

In [7]:
a.memory

[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'},
 {'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'},
 {'role': 'user', 'content': 'hi'},
 {'role': 'user', 'content': '换成小羊吧'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'},
 {'role': 'user', 'content': '改为在蓝天上'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'},
 {'role': 'user', 'content': '改为在水塘'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]